# COMP9318 Project
## Exploratory Data Analysis

In [2]:
import helper as h
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas2arff

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

Read in word data:

In [3]:
data_loc = 'asset/training_data.txt'
words = h.get_words(data_loc)

In [4]:
words.head()

word                pronunciation  \
0          COED                  K OW1 EH2 D   
1       PURVIEW                P ER1 V Y UW2   
2         HEHIR              HH EH1 HH IH0 R   
3      MUSCLING         M AH1 S AH0 L IH0 NG   
4  NONPOISONOUS  N AA0 N P OY1 Z AH0 N AH0 S   

                                  pn_list                  destressed_pn_list  \
0                        [K, OW1, EH2, D]                      [K, OW, EH, D]   
1                     [P, ER1, V, Y, UW2]                   [P, ER, V, Y, UW]   
2                   [HH, EH1, HH, IH0, R]                 [HH, EH, HH, IH, R]   
3            [M, AH1, S, AH0, L, IH0, NG]           [M, AH, S, AH, L, IH, NG]   
4  [N, AA0, N, P, OY1, Z, AH0, N, AH0, S]  [N, AA, N, P, OY, Z, AH, N, AH, S]   

               primary_stress_map            secondary_stress_map  \
0                    [0, 1, 0, 0]                    [0, 0, 1, 0]   
1                 [0, 1, 0, 0, 0]                 [0, 0, 0, 0, 1]   
2                 [0, 1, 0, 0, 0]                 [0, 0, 0, 0, 0]   
3           [0, 1, 0, 0, 0, 0, 0]           [0, 0, 0, 0, 0, 0, 0]   
4  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   

                        vowel_map                   consonant_map  \
0                    [0, 1, 1, 0]                    [1, 0, 0, 1]   
1                 [0, 1, 0, 0, 1]                 [1, 0, 1, 1, 0]   
2                 [0, 1, 0, 1, 0]                 [1, 0, 1, 0, 1]   
3           [0, 1, 0, 1, 0, 1, 0]           [1, 0, 1, 0, 1, 0, 1]   
4  [0, 1, 0, 0, 1, 0, 1, 0, 1, 0]  [1, 0, 1, 1, 0, 1, 0, 1, 0, 1]   

                                          vector_map  vowel_count ...  R  S  \
0  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...            2 ...  0  0   
1  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, ...            2 ...  0  0   
2  [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...            2 ...  1  0   
3  [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...            3 ...  0  1   
4  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...            4 ...  0  1   

   SH  T  TH  V  W  Y  Z  ZH  
0   0  0   0  0  0  0  0   0  
1   0  0   0  1  0  1  0   0  
2   0  0   0  0  0  0  0   0  
3   0  0   0  0  0  0  0   0  
4   0  0   0  0  0  0  1   0  

[5 rows x 57 columns]

In [5]:
words.columns

Index(['word', 'pronunciation', 'pn_list', 'destressed_pn_list',
       'primary_stress_map', 'secondary_stress_map', 'vowel_map',
       'consonant_map', 'vector_map', 'vowel_count', 'consonant_count',
       'type_tag', '1st_letter_idx', 'phoneme_length', 'prefix', 'suffix',
       'primary_stress_idx', 'stressed_vowel', 'AA', 'AE', 'AH', 'AO', 'AW',
       'AY', 'EH', 'ER', 'EY', 'IH', 'IY', 'OW', 'OY', 'UH', 'UW', 'P', 'B',
       'CH', 'D', 'DH', 'F', 'G', 'HH', 'JH', 'K', 'L', 'M', 'N', 'NG', 'R',
       'S', 'SH', 'T', 'TH', 'V', 'W', 'Y', 'Z', 'ZH'],
      dtype='object')

### Facts

Most common stressed vowel in descending order. Compare against total count of particular vowel, can then take proability that vowel will be stressed

In [6]:
# Get count where particular vowel is the stress vowel and count where vowel is in existance
stressed_vowel_count = words.groupby('stressed_vowel')['word'].count().sort_values(ascending=False)
total_vowel_count = words[list(h.vowels)].sum()

# Join the two frames
vowel_counts = pd.concat((stressed_vowel_count,total_vowel_count), axis=1, join='inner')
vowel_counts.columns = ['stressed_vowel_count','total_vowel_count']
vowel_counts['vowel_count_prob'] = vowel_counts.stressed_vowel_count/vowel_counts.total_vowel_count
vowel_counts.sort_values(['vowel_count_prob'],ascending=False)

stressed_vowel_count  total_vowel_count  vowel_count_prob
AE                  6442               8104          0.794916
EH                  7892              10090          0.782161
AA                  6331               8957          0.706821
AO                  2952               4260          0.692958
EY                  3167               4728          0.669839
OY                   297                446          0.665919
UW                  2328               3522          0.660988
AW                   853               1308          0.652141
UH                   608                933          0.651661
AY                  2502               4178          0.598851
OW                  3185               7092          0.449098
IH                  5805              16630          0.349068
IY                  3515              12281          0.286214
ER                  1645              11506          0.142969
AH                  2478              23623          0.104898

Find patterns in training data, build list of all possible consecutive sequences

In [62]:
def sub_string(pronunciation_list,length):
    return tuple(zip(*(pronunciation_list[i:] for i in range(length))))

def get_sequences(phoneme_series):
    sequences = {}
    max_length = max(phoneme_series.str.len())
    for i in range(2, max_length + 1):
        for pn_list in phoneme_series:
            # Next iteration if pn_list is shorter then the sequence length be built
            if len(pn_list) < i:
                continue
            word_sequences = sub_string(pn_list,i)
            for seq in word_sequences:
                sequences[seq] = sequences.get(seq,0) + 1
    return sequences

def in_list(pn_list,sequence):
    if pn_list in sequence:
        return 1
    return 0

def is_primary(sequence):
    for phoneme in sequence:
        if '1' in phoneme:
            return True
    return False

# Generate Dataframe with all destressed sequence possibilities and get counts
destressed_sequences = get_sequences(words.destressed_pn_list)
destressed_sequence_df = pd.DataFrame(list(destressed_sequences.items()),columns=['Destressed_Sequence','Destressed_Sequence_Count'])
destressed_sequence_df = destressed_sequence_df.set_index('Destressed_Sequence')
# Generate Dataframe with all sequence possibiities and get counts, flag if primary stress in sequence
sequences = get_sequences(words.pn_list)
sequence_df = pd.DataFrame(list(sequences.items()),columns=['Sequence','Sequence_Count'])

# Return True is sequence has primary stress in it
sequence_df['Is_Primary'] = sequence_df.Sequence.apply(is_primary)
sequence_df['Destressed_Sequence'] = sequence_df.Sequence.apply(h.filter_stress)
sequence_df.Destressed_Sequence = sequence_df.Destressed_Sequence.apply(h.as_tuple)
sequence_df = sequence_df.query('Is_Primary == True').set_index('Destressed_Sequence')

# Join 
sequences_df = sequence_df.join(destressed_sequence_df)

# Get probability that sequence if exists will be stressed
sequences_df['Sequence_Stress_Probability'] = sequences_df.Sequence_Count/sequences_df.Destressed_Sequence_Count
sequences_df['Sequence_Length'] = sequences_df.Sequence.str.len()

In [59]:
destressed_sequence_df

Destressed_Sequence_Count
Destressed_Sequence                                                        
(K, OW)                                                                 858
(OW, EH)                                                                 15
(EH, D)                                                                 541
(P, ER)                                                                 712
(ER, V)                                                                 127
(V, Y)                                                                   38
(Y, UW)                                                                 811
(HH, EH)                                                                432
(EH, HH)                                                                 25
(HH, IH)                                                                276
(IH, R)                                                                 607
(M, AH)                                                                3006
(AH, S)                                                                2055
(S, AH)                                                                2052
(AH, L)                                                                4631
(L, IH)                                                                2016
(IH, NG)                                                               3245
(N, AA)                                                                 425
(AA, N)                                                                1609
(N, P)                                                                   50
(P, OY)                                                                  43
(OY, Z)                                                                  20
(Z, AH)                                                                 631
(AH, N)                                                                8526
(N, AH)                                                                1757
(L, AA)                                                                 646
(AA, V)                                                                 215
(V, EH)                                                                 309
(EH, K)                                                                1078
(K, IY)                                                                 997
...                                                                     ...
(R, IY, K, AH, N, S, T, R, AH, K, T, IH, NG)                              1
(S, T, R, AE, NG, G, Y, AH, L, EY, SH, AH, N)                             1
(T, R, AE, NG, G, Y, AH, L, EY, SH, AH, N, Z)                             1
(K, AA, NG, G, R, AH, S, P, ER, S, AH, N, Z)                              1
(P, R, IY, K, AH, N, S, EH, P, SH, AH, N, Z)                              1
(S, AH, B, K, AA, N, T, R, AE, K, T, ER, Z)                               1
(D, IH, S, T, R, AH, K, T, IH, V, N, IH, S)                               1
(N, AA, N, R, IH, S, T, R, IH, K, T, IH, D)                               1
(D, IH, S, R, IH, S, P, EH, K, T, F, AH, L)                               1
(N, AA, N, K, AH, M, B, AE, T, AH, N, T, S)                               1
(S, W, AO, R, T, S, AH, N, T, R, UW, B, ER)                               1
(D, IH, S, IH, N, F, R, AE, N, CH, AY, Z, D)                              1
(S, AH, B, S, T, IH, T, Y, UW, SH, AH, N, Z)                              1
(T, R, AE, N, S, EH, K, SH, Y, UW, AH, L, Z)                              1
(K, AA, N, S, AH, N, T, R, EY, SH, AH, N, Z)                              1
(P, R, AA, P, AH, G, AE, N, D, AH, S, T, S)                               1
(N, AA, T, W, IH, TH, S, T, AE, N, D, IH, NG)                             1
(N, AA, N, P, R, AH, S, K, R, IH, P, SH, AH)                              1
(AA, N, P, R, AH, S, K, R, IH, P, SH, AH, N)                              1
(P, R, AH, T, EH, K, SH, AH, N, IH, S, T, S)                              1
(IH, N, S, T, R, AH, M

In [8]:
valid_seqs = sequences_df.query('Sequence_Stress_Probability  < 1 & Destressed_Sequence_Count > 5')
valid_seqs.sort_values(by=['Sequence_Stress_Probability','Destressed_Sequence_Count'],ascending=False)
#sequences_df.query('Sequence_Stress_Probability > 0.5  & Destressed_Sequence_Count > 5').sort_values(by='Destressed_Sequence',ascending=False)
sequences_df.query('Sequence_Stress_Probability > 0.5  & Sequence_Length < 3')

# Doorknob
sequences_df.loc[[('UW','R'),('AA','B')]]

# Rabon
#RABON:R AA0 B AO1 N
sequences_df.loc[[('AA','B'),('AO','N')]]

#ABILA:AA0 B IY1 L AH0
sequences_df.loc[[('AA','B'),('IY','L')]]

Sequence  Sequence_Count Is_Primary  \
Destressed_Sequence                                        
(AA, B)              (AA1, B)             182       True   
(IY, L)              (IY1, L)             276       True   

                     Destressed_Sequence_Count  Sequence_Stress_Probability  \
Destressed_Sequence                                                           
(AA, B)                                    239                     0.761506   
(IY, L)                                    506                     0.545455   

                     Sequence_Length  
Destressed_Sequence                   
(AA, B)                            2  
(IY, L)                            2


Only show sequences that appear in atleast 1% of words in training data. Compare sequence with stresses included.

In [35]:
sequences_df.sort_values(by='Sequence',ascending=True)

Sequence  Sequence_Count  \
Destressed_Sequence                                                        
(AA, AA)                                      (AA0, AA1)               1   
(AA, AA, K)                                (AA0, AA1, K)               1   
(AA, AA, K, IY)                       (AA0, AA1, K, IY2)               1   
(AA, B, AA, L, D, AO)           (AA0, B, AA0, L, D, AO1)               1   
(AA, B, AA, L, D, AO, N)     (AA0, B, AA0, L, D, AO1, N)               1   
(AA, B, AA, T, IY)                 (AA0, B, AA0, T, IY1)               1   
(AA, B, AA, T, IY, N)           (AA0, B, AA0, T, IY1, N)               1   
(AA, B, AA, T, IY, N, IY)  (AA0, B, AA0, T, IY1, N, IY0)               1   
(AA, B, AA)                                (AA0, B, AA1)               6   
(AA, B, AA, G)                          (AA0, B, AA1, G)               1   
(AA, B, AA, G, OW)                 (AA0, B, AA1, G, OW0)               1   
(AA, B, AA, L)                          (AA0, B, AA1, L)               1   
(AA, B, AA, L, OW)                 (AA0, B, AA1, L, OW0)               1   
(AA, B, AA, L, OW, Z)           (AA0, B, AA1, L, OW0, Z)               1   
(AA, B, AA, R)                          (AA0, B, AA1, R)               3   
(AA, B, AA, R, EH)                 (AA0, B, AA1, R, EH0)               1   
(AA, B, AA, R, EH, S)           (AA0, B, AA1, R, EH0, S)               1   
(AA, B, AA, R, IY)                 (AA0, B, AA1, R, IY0)               1   
(AA, B, AA, W)                          (AA0, B, AA1, W)               1   
(AA, B, AA, W, AH)                 (AA0, B, AA1, W, AH0)               1   
(AA, B, AO)                                (AA0, B, AO1)               2   
(AA, B, AO, N)                          (AA0, B, AO1, N)               2   
(AA, B, D, UW)                          (AA0, B, D, UW1)               1   
(AA, B, D, UW, L)                    (AA0, B, D, UW1, L)               1   
(AA, B, D, UW, L, AH)           (AA0, B, D, UW1, L, AH0)               1   
(AA, B, EH)                                (AA0, B, EH1)               7   
(AA, B, EH, L)                          (AA0, B, EH1, L)               5   
(AA, B, EH, L, AH)                 (AA0, B, EH1, L, AH0)               1   
(AA, B, EH, L, IY)                 (AA0, B, EH1, L, IY0)               1   
(AA, B, EH, N)                          (AA0, B, EH1, N)               1   
...                                                  ...             ...   
(ZH, IH, L, M, UH, R)            (ZH, IH0, L, M, UH1, R)               1   
(ZH, IH, N, D, R, OW)            (ZH, IH0, N, D, R, OW1)               1   
(ZH, IH, N, EH)                        (ZH, IH0, N, EH1)               1   
(ZH, IH, N, EH, T)                  (ZH, IH0, N, EH1, T)               1   
(ZH, IH, V, AA)                        (ZH, IH0, V, AA1)               1   
(ZH, IH, V, AA, G)                  (ZH, IH0, V, AA1, G)               1   
(ZH, IH, V, AA, G, OW)         (ZH, IH0, V, AA1, G, OW0)               1   
(ZH, IH)                                       (ZH, IH1)               3   
(ZH, IH, N)                                 (ZH, IH1, N)               1   
(ZH, IH, N, S)                           (ZH, IH1, N, S)               1   
(ZH, IH, N, S, K)                     (ZH, IH1, N, S, K)               1   
(ZH, IH, N, S, K, IY)            (ZH, IH1, N, S, K, IY0)               1   
(ZH, IH, NG)                               (ZH, IH1, NG)               1   
(ZH, IH, R)                                 (ZH, IH1, R)               1   
(ZH, IH, R, AA)                        (ZH, IH1, R, AA2)               1   
(ZH, IH, R, AA, R)                  (ZH, IH1, R, AA2, R)               1   
(ZH, IH, R, AA, R, D)            (ZH, IH1, R, AA2, R, D)               1   
(ZH, IH, R, AA, R, D, OW)   (ZH, IH1, R, AA2, R, D, OW0)               1   
(ZH, IY, AE)                              (ZH, IY0, AE1)               1   
(ZH, IY, AE, NG)                      (ZH, IY0, AE1, NG)               1   
(ZH, IY)            

Remove longer super sequences that can be described by a existing sub sequence

In [56]:
def has_subsequence(sequence,sequence_set):
    # Do not check sequences of length 2 or the final as they will obviously be in the set
    for i in range(2,len(sequence)):
        subsequence = sequence[0:i]
        if subsequence in sequence_set :
            return True
    return False

sequences_df['Has_Subsequence'] = sequences_df.Sequence.apply(has_subsequence,args=(set(sequences_df.Sequence),))
sequences_df

Sequence  Sequence_Count  \
Destressed_Sequence                                                       
(AA, AA)                                     (AA0, AA1)               1   
(AA, AA, K)                               (AA0, AA1, K)               1   
(AA, AA, K, IY)                      (AA0, AA1, K, IY2)               1   
(AA, AH)                                     (AA1, AH0)               5   
(AA, AH, L)                               (AA1, AH0, L)               1   
(AA, AH, M)                               (AA1, AH0, M)               1   
(AA, AH, N)                               (AA1, AH0, N)               2   
(AA, AH, N, IH)                      (AA1, AH0, N, IH0)               2   
(AA, AH, N, IH, NG)              (AA1, AH0, N, IH0, NG)               2   
(AA, AH, S)                               (AA1, AH0, S)               1   
(AA, AH, S, T)                         (AA1, AH0, S, T)               1   
(AA, AO)                                     (AA1, AO0)               1   
(AA, AO, G)                               (AA1, AO0, G)               1   
(AA, AO, G, W)                         (AA1, AO0, G, W)               1   
(AA, AO, G, W, AE)                (AA1, AO0, G, W, AE2)               1   
(AA, AO, G, W, AE, NG)        (AA1, AO0, G, W, AE2, NG)               1   
(AA, AY)                                     (AA1, AY0)               1   
(AA, B)                                        (AA1, B)             182   
(AA, B, AA)                               (AA0, B, AA1)               6   
(AA, B, AA)                               (AA1, B, AA0)               4   
(AA, B, AA)                               (AA2, B, AA1)               1   
(AA, B, AA, G)                         (AA0, B, AA1, G)               1   
(AA, B, AA, G, OW)                (AA0, B, AA1, G, OW0)               1   
(AA, B, AA, L)                         (AA0, B, AA1, L)               1   
(AA, B, AA, L, D, AO)          (AA0, B, AA0, L, D, AO1)               1   
(AA, B, AA, L, D, AO, N)    (AA0, B, AA0, L, D, AO1, N)               1   
(AA, B, AA, L, OW)                (AA0, B, AA1, L, OW0)               1   
(AA, B, AA, L, OW, Z)          (AA0, B, AA1, L, OW0, Z)               1   
(AA, B, AA, R)                         (AA0, B, AA1, R)               3   
(AA, B, AA, R)                         (AA1, B, AA0, R)               1   
...                                                 ...             ...   
(ZH, IH, L, M, UH)                 (ZH, IH0, L, M, UH1)               1   
(ZH, IH, L, M, UH, R)           (ZH, IH0, L, M, UH1, R)               1   
(ZH, IH, N)                                (ZH, IH1, N)               1   
(ZH, IH, N, D, R, OW)           (ZH, IH0, N, D, R, OW1)               1   
(ZH, IH, N, EH)                       (ZH, IH0, N, EH1)               1   
(ZH, IH, N, EH, T)                 (ZH, IH0, N, EH1, T)               1   
(ZH, IH, N, S)                          (ZH, IH1, N, S)               1   
(ZH, IH, N, S, K)                    (ZH, IH1, N, S, K)               1   
(ZH, IH, N, S, K, IY)           (ZH, IH1, N, S, K, IY0)               1   
(ZH, IH, NG)                              (ZH, IH1, NG)               1   
(ZH, IH, R)                                (ZH, IH1, R)               1   
(ZH, IH, R, AA)                       (ZH, IH1, R, AA2)               1   
(ZH, IH, R, AA, R)                 (ZH, IH1, R, AA2, R)               1   
(ZH, IH, R, AA, R, D)           (ZH, IH1, R, AA2, R, D)               1   
(ZH, IH, R, AA, R, D, OW)  (ZH, IH1, R, AA2, R, D, OW0)               1   
(ZH, IH, V, AA)                       (ZH, IH0, V, AA1)               1   
(ZH, IH, V, AA, G)                 (ZH, IH0, V, AA1, G)               1   
(ZH, IH, V, AA, G, OW)        (ZH, IH0, V, AA1, G, OW0)               1   
(ZH, IY)                                      (ZH, IY1)               2   
(ZH, IY, AE)                             (ZH, IY0, AE1)               1   
(ZH, IY, AE, NG)                     (ZH, IY0, AE1, NG)               1 

### Plots

In [ ]:
%matplotlib inline


def scatter_plot(x,y):
    fig = plt.figure()
    ax  = fig.add_subplot(1,1,1)
    ax.scatter(x,y)
    plt.title("{} vs {}".format(y.name,x.name))
    plt.xlabel(x.name)
    plt.ylabel(y.name)
    plt.show()
    return

def histogram(data):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.hist(data,bins='auto')
    plt.title('{} Distribution'.format(data.name))
    plt.xlabel('{}'.format(data.name))
    #plt.xlim(0,bins_count)
    plt.show()
    return    

In [ ]:
for tag in words.type_tag.unique():
    tag_df = words[words['type_tag'] == tag]
    print(tag)
    scatter_plot(tag_df['1st_letter_idx'], tag_df.primary_stress_idx)

In [ ]:
histogram(words.primary_stress_idx.apply(int))

In [ ]:
histogram(sequence_df[sequence_df.Sequence_Count > 0].Sequence_Count)